# Import Danych z Senuto do Supabase

Ten skrypt umożliwia automatyczne zaimportowanie danych z wyeksportowanego raportu Excel z narzędzia Senuto bezpośrednio do Twojej bazy danych Supabase. Jest to uproszczona wersja, która skupia się na kluczowych metrykach słów kluczowych i przypisaniu ich do adresów URL.

**Wymagania wstępne:**

1.  **Projekt Supabase:** Posiadanie aktywnego projektu w Supabase.
2.  **Tabele w Supabase:** W Twojej bazie danych muszą istnieć dwie tabele o nazwach (przypad ma znaczenie!):
    *   `url_data`: Powinna zawierać kolumny takie jak:
        *   `id` (klucz główny, typ `int8` lub podobny, najlepiej z `identity` lub sequence dla autoinkrementacji)
        *   `url` (typ `text`, **unikalny**)
        *   Opcjonalnie: `title`, `description` (typ `text`) - skrypt ustawi je na `NULL`, ale kolumny muszą istnieć.
    *   `master_data`: Powinna zawierać kolumny takie jak:
        *   `id` (klucz główny, autoinkrementowany)
        *   `url_id` (typ `int8` lub podobny, klucz obcy odwołujący się do `url_data.id`)
        *   `keyword` (typ `text`)
        *   `avg_monthly_searches`, `position`, `previous_position`, `position_change`, `keyword_difficulty` (typ `int8` lub inny liczbowy całkowity)
        *   `estimated_traffic`, `previous_estimated_traffic`, `traffic_change`, `cpc` (typ `float8` lub inny liczbowy zmiennoprzecinkowy)
3.  **Raport Senuto:** Plik Excel wygenerowany z raportu Senuto, zawierający standardowe kolumny, w szczególności: 'Słowo kluczowe', 'Śr. mies. liczba wyszukiwań', 'Pozycja', 'Pozycja (poprzednia)', 'Zmiana pozycji', 'Szacowany ruch', 'Szacowany ruch (poprzednio)', 'Szacowany ruch (zmiana)', 'CPC', 'Trudność słowa kluczowego', 'Adres URL', 'Adres URL (poprzedni)'. Raport z tego raportu: https://app.senuto.com/visibility-analysis/positions?domain=smyk.com&fetch_mode=subdomain&country_id=200
4.  **Biblioteka Supabase:** Upewnij się, że w środowisku Colab zainstalowana jest biblioteka `supabase` (`!pip install supabase`). Zazwyczaj jest dostępna domyślnie.

**Jak użyć:**

1.  Uruchom tę komórkę kodu w Google Colab.
2.  W polach formularza, które pojawią się pod kodem, wprowadź swój `SUPABASE_URL` i `SUPABASE_KEY` (znajdziesz je w ustawieniach projektu Supabase w sekcji "API"). Możesz także dostosować `batch_size` (rozmiar partii danych wysyłanych jednorazowo do bazy).
3.  Skrypt poprosi Cię o wgranie pliku Excel z raportem Senuto. Kliknij przycisk "Wybierz pliki" i wskaż odpowiedni plik na swoim komputerze.
4.  Skrypt przetworzy dane:
    *   Zmapuje polskie nazwy kolumn na angielskie (zgodne z wymaganymi nazwami kolumn w Twoich tabelach Supabase).
    *   Wyczyści dane (zamieni puste wartości na `NULL`, usunie białe znaki).
    *   Wyodrębni unikalne adresy URL i zaimportuje je do tabeli `url_data` używając operacji `UPSERT` (co oznacza, że jeśli URL już istnieje, zostanie zaktualizowany/pominięty w zależności od konfiguracji Supabase, a jeśli nie, zostanie dodany). Skrypt pobierze następnie ID dodanych/istniejących URL-i.
    *   Przygotuje główne dane słów kluczowych, łącząc je z ID odpowiednich URL-i.
    *   Zaimportuje dane słów kluczowych do tabeli `master_data` używając operacji `INSERT` (każde uruchomienie skryptu z tym samym raportem doda nowe rekordy do `master_data`, jeśli nie masz odpowiednich ograniczeń unikalności w bazie).
5.  Na końcu skrypt zapyta, czy chcesz zweryfikować import. Wpisz `tak` i naciśnij Enter, aby wyświetlić liczbę zaimportowanych rekordów i kilka przykładów.

**Uwagi:**

*   Skrypt zakłada, że raport Senuto ma standardową strukturę kolumn. Jeśli nazwy kolumn są inne, musisz zmodyfikować słownik `column_mapping` w skrypcie.
*   Konwersja typów danych (`int` i `float`) jest zaimplementowana, ale upewnij się, że dane w Excelu są w czystej formie, aby uniknąć błędów podczas konwersji.
*   Pola `title` i `description` w tabeli `url_data` nie są wypełniane przez ten skrypt - są ustawiane na `NULL`.
*   Import do `master_data` używa `INSERT`, co oznacza, że jeśli uruchomisz skrypt wielokrotnie z tym samym plikiem, dodasz duplikaty do tej tabeli (chyba że w bazie masz ustawione ograniczenia unikalności).

In [ ]:
# Import danych z Senuto do Supabase - uproszczona wersja z poprawioną konwersją typów

import pandas as pd
import numpy as np
from supabase import create_client
import io
from datetime import datetime
from google.colab import files

# Konfiguracja połączenia do Supabase
#@markdown Szukaj tych danych w ustawieniach projektu w Supabase
SUPABASE_URL = "" #@param {type:"string"}
SUPABASE_KEY = "" #@param {type:"string"}

# Inicjalizacja klienta Supabase
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

# 1. Wczytywanie pliku Excel
print("Wgraj plik Excel z raportem Senuto:")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"Wczytywanie pliku: {file_name}")

# Wczytanie pliku Excel
df = pd.read_excel(io.BytesIO(uploaded[file_name]))
print(f"Wczytano plik z {len(df)} wierszami.")

# 2. Mapowanie kolumn z polskiego na angielski
column_mapping = {
    'Słowo kluczowe': 'keyword',
    'Śr. mies. liczba wyszukiwań': 'avg_monthly_searches',
    'Pozycja': 'position',
    'Pozycja (poprzednia)': 'previous_position',
    'Zmiana pozycji': 'position_change',
    'Szacowany ruch': 'estimated_traffic',
    'Szacowany ruch (poprzednio)': 'previous_estimated_traffic',
    'Szacowany ruch (zmiana)': 'traffic_change',
    'CPC': 'cpc',
    'Trudność słowa kluczowego': 'keyword_difficulty',
    'Adres URL': 'url',
    'Adres URL (poprzedni)': 'previous_url'
}
df = df.rename(columns=column_mapping)

# 3. Czyszczenie danych
# Zamiana pustych wartości na NULL
df = df.replace('', None)
df = df.replace(np.nan, None)

# Przycinanie białych znaków z tekstu
string_columns = ['keyword', 'url', 'previous_url']
for col in string_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# 4. Ekstrakcja unikalnych URL-i
urls = df['url'].dropna().unique()
url_data = []
for url in urls:
    url_data.append({
        'url': url,
        'title': None,
        'description': None
    })

print(f"Znaleziono {len(url_data)} unikalnych URL-i.")

# 5. Import URL-i do url_data
print("Importowanie URL-i do tabeli url_data...")
url_to_id = {}  # Mapowanie URL -> id

# Importuj URL-e w małych partiach
#@markdown Rozmiar partii danych w jakich skrypt ma upsertować dane do bazy danych
batch_size = 30 #@param {type:"integer"}
for i in range(0, len(url_data), batch_size):
    batch = url_data[i:i+batch_size]
    try:
        result = supabase.table('url_data').upsert(batch, on_conflict='url').execute()
        print(f"  Zaimportowano {len(batch)} URL-i (partia {i//batch_size + 1}/{(len(url_data)+batch_size-1)//batch_size})")
    except Exception as e:
        print(f"  Błąd podczas importu URL-i: {e}")
        for url_item in batch:
            try:
                result = supabase.table('url_data').upsert([url_item], on_conflict='url').execute()
                print(f"    Zaimportowano URL: {url_item['url']}")
            except Exception as e2:
                print(f"    Błąd importu URL {url_item['url']}: {e2}")

# 6. Pobierz ID-ki zaimportowanych URL-i
for url in urls:
    try:
        result = supabase.table('url_data').select('id').eq('url', url).execute()
        if result.data and len(result.data) > 0:
            url_to_id[url] = result.data[0]['id']
    except Exception as e:
        print(f"  Błąd podczas pobierania ID dla URL {url}: {e}")

print(f"Pobrano {len(url_to_id)} ID-ków URL-i.")

# 7. Przygotowanie danych do tabeli master_data
master_data = []
for _, row in df.iterrows():
    # Tylko jeśli znamy URL ID
    if row['url'] in url_to_id:
        record = {
            'keyword': row['keyword'],
            'url_id': int(url_to_id[row['url']])
        }

        # Kolumny typu INTEGER
        int_cols = ['avg_monthly_searches', 'position', 'previous_position',
                   'position_change', 'keyword_difficulty']
        for col in int_cols:
            if col in row and pd.notna(row[col]):
                # Upewniamy się, że wartość jest typu int
                try:
                    record[col] = int(float(row[col]))
                except:
                    record[col] = 0
            else:
                record[col] = 0

        # Kolumny typu FLOAT
        float_cols = ['estimated_traffic', 'previous_estimated_traffic',
                     'traffic_change', 'cpc']
        for col in float_cols:
            if col in row and pd.notna(row[col]):
                # Upewniamy się, że wartość jest typu float
                try:
                    record[col] = float(row[col])
                except:
                    record[col] = 0.0
            else:
                record[col] = 0.0

        master_data.append(record)

print(f"Przygotowano {len(master_data)} rekordów do importu do tabeli master_data.")

# 8. Import danych do master_data
print("Importowanie danych do tabeli master_data...")

# Importuj w małych partiach
for i in range(0, len(master_data), batch_size):
    batch = master_data[i:i+batch_size]
    try:
        result = supabase.table('master_data').insert(batch).execute()
        print(f"  Zaimportowano {len(batch)} rekordów (partia {i//batch_size + 1}/{(len(master_data)+batch_size-1)//batch_size})")
    except Exception as e:
        print(f"  Błąd podczas importu danych: {e}")
        # Spróbuj po jednym rekordzie
        for record in batch:
            try:
                result = supabase.table('master_data').insert([record]).execute()
                print(f"    Zaimportowano rekord dla słowa kluczowego: {record['keyword']}")
            except Exception as e2:
                print(f"    Błąd importu rekordu dla słowa kluczowego {record['keyword']}: {e2}")
                print(f"    Dane rekordu: {record}")

print("Import danych zakończony.")

# 9. Weryfikacja importu
def verify_import():
    print("\nWeryfikacja zaimportowanych danych:")

    try:
        # Sprawdź liczbę zaimportowanych URL-i
        result = supabase.table('url_data').select('count').execute()
        url_count = result.data[0]['count'] if result.data and 'count' in result.data[0] else 0
        print(f"Liczba URL-i w tabeli url_data: {url_count}")

        # Sprawdź liczbę zaimportowanych rekordów
        result = supabase.table('master_data').select('count').execute()
        master_count = result.data[0]['count'] if result.data and 'count' in result.data[0] else 0
        print(f"Liczba rekordów w tabeli master_data: {master_count}")

        # Pokaż kilka przykładowych rekordów URL
        result = supabase.table('url_data').select('*').limit(3).execute()
        print("\nPrzykładowe rekordy URL:")
        for record in result.data:
            print(f"  ID: {record['id']}, URL: {record['url']}")

        # Pokaż kilka przykładowych rekordów master_data
        result = supabase.table('master_data').select('*').limit(3).execute()
        print("\nPrzykładowe rekordy master_data:")
        for record in result.data:
            print(f"  ID: {record['id']}, Keyword: {record['keyword']}, URL ID: {record['url_id']}")

    except Exception as e:
        print(f"Błąd podczas weryfikacji: {e}")

# Zapytaj o weryfikację
verify_confirm = input("\nCzy chcesz zweryfikować zaimportowane dane? (tak/nie): ")
if verify_confirm.lower() == 'tak':
    verify_import()

print("\nProces importu zakończony.")

Wgraj plik Excel z raportem Senuto:


Saving analiza_widoczno_ci_raport__senuto_com_2025_04_30_15_20 (1).xlsx to analiza_widoczno_ci_raport__senuto_com_2025_04_30_15_20 (1) (4).xlsx
Wczytywanie pliku: analiza_widoczno_ci_raport__senuto_com_2025_04_30_15_20 (1) (4).xlsx
Wczytano plik z 4103 wierszami.
Znaleziono 1065 unikalnych URL-i.
Importowanie URL-i do tabeli url_data...
  Zaimportowano 10 URL-i (partia 1/107)
  Zaimportowano 10 URL-i (partia 2/107)
  Zaimportowano 10 URL-i (partia 3/107)
  Zaimportowano 10 URL-i (partia 4/107)
  Zaimportowano 10 URL-i (partia 5/107)
  Zaimportowano 10 URL-i (partia 6/107)
  Zaimportowano 10 URL-i (partia 7/107)
  Zaimportowano 10 URL-i (partia 8/107)
  Zaimportowano 10 URL-i (partia 9/107)
  Zaimportowano 10 URL-i (partia 10/107)
  Zaimportowano 10 URL-i (partia 11/107)
  Zaimportowano 10 URL-i (partia 12/107)
  Zaimportowano 10 URL-i (partia 13/107)
  Zaimportowano 10 URL-i (partia 14/107)
  Zaimportowano 10 URL-i (partia 15/107)
  Zaimportowano 10 URL-i (partia 16/107)
  Zaimportowan